# Pokemon Type Guesser

### importing general modules

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

### getting the data

In [2]:
data=pd.read_csv('pokemon.csv')

In [3]:
data.head()

,Name,Type1,Type2,Evolution
0,bulbasaur,Grass,Poison,ivysaur
1,ivysaur,Grass,Poison,venusaur
2,venusaur,Grass,Poison,NaN
3,charmander,Fire,NaN,charmeleon
4,charmeleon,Fire,NaN,charizard


#### getting to know about data

In [4]:
data.describe()

,Name,Type1,Type2,Evolution
count,809,809,405,32
unique,809,18,18,32
top,bulbasaur,Water,Flying,ivysaur
freq,1,114,95,1


In [5]:
data.isnull().sum()

Name           0
Type1          0
Type2        404
Evolution    777
dtype: int64

#### croppping the data 

In [6]:
data_fil=data.drop(columns=['Type2','Evolution'],axis=1)

In [7]:
data_fil.head()

,Name,Type1
0,bulbasaur,Grass
1,ivysaur,Grass
2,venusaur,Grass
3,charmander,Fire
4,charmeleon,Fire


## changing the name field to image path

In [8]:
data_fil['Name']="images" + "\\"+data_fil['Name']+'.png'

In [9]:
data_fil.head()

,Name,Type1
0,images\bulbasaur.png,Grass
1,images\ivysaur.png,Grass
2,images\venusaur.png,Grass
3,images\charmander.png,Fire
4,images\charmeleon.png,Fire


##### partioning to the X and y set

In [10]:
x=data_fil['Name']
y=data_fil['Type1']

In [11]:
y.head()

0    Grass
1    Grass
2    Grass
3     Fire
4     Fire
Name: Type1, dtype: object

#### Red-Hot Encoding the type field  

In [12]:

from sklearn.preprocessing import LabelEncoder

y_end=LabelEncoder().fit_transform(y)
y_end[:10]

array([ 9,  9,  9,  6,  6,  6, 17, 17, 17,  0])

#### Splitting into test and train data 

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y_end,test_size=0.2,random_state=42)
print(type(xtrain))
# print(xtrain[0].shape)

<class 'pandas.core.series.Series'>


#### converting image to array

In [14]:
from tensorflow.keras.preprocessing import image


def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array
xtrain = np.array([load_and_preprocess_image(img) for img in xtrain])
xtest = np.array([load_and_preprocess_image(img) for img in xtest])
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)
ytest=np.array(ytest)
xtest=np.array(xtest)


#### shuffling the data and making data to train and test

In [15]:

indices=np.arange(xtrain.shape[0])
np.random.shuffle(indices)
xtrain=xtrain[indices]
ytrain=ytrain[indices]

traindata=tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
traindata=traindata.shuffle(buffer_size=1024).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valdata=tf.data.Dataset.from_tensor_slices((xtest,ytest))
valdata=valdata.shuffle(buffer_size=1024).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


#### Building the model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report , accuracy_score , precision_score , recall_score

input_shape = (224, 224, 3)
# Create the input layer
input_layer = Input(shape=input_shape)

# Add layers to build the model
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(18, activation='softmax')(x)  # Correct the output layer

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy')
history = model.fit(traindata,epochs=25,validation_data=valdata,verbose=1)



Epoch 1/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 633ms/step - loss: 2.8566 - val_loss: 2.8002
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 610ms/step - loss: 2.6914 - val_loss: 2.7559
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 625ms/step - loss: 2.4798 - val_loss: 2.8175
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 617ms/step - loss: 2.0479 - val_loss: 2.9218
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 629ms/step - loss: 1.3827 - val_loss: 3.1165
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 649ms/step - loss: 1.1244 - val_loss: 3.5116
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 614ms/step - loss: 0.7393 - val_loss: 4.3434
Epoch 8/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 607ms/step - loss: 0.5857 - val_loss: 4.2532
Epoch 9/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 637ms/step - loss: 0.3874 - val_loss: 5.3046
Epoch 10/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 635ms/step - loss: 0.2725 - val_loss: 4.8627
Epoch 11/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 650ms/step - loss: 0.2233 - val_loss: 5.5931
Epoch 12/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 13